<a href="https://colab.research.google.com/github/abdullahnizami77/inshorts-scraping/blob/main/inshorts_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import urllib
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import quote
import os
import time
import sys
import random
import string
import re

def myquote(quote_page):
    url = urllib.parse.urlsplit(quote_page)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    return url

try:
    if "hindi" in str(sys.argv[1]).lower():
        file_name = "/content/links.txt"
        destination_path = "/content/dataset/hindi/"
        full_ds_path = "/content/inshorts-dataset-hi/"
    else:
        file_name = "/content/links.txt"
        destination_path = "/content/dataset/english/"
        full_ds_path = "/content/inshorts-dataset-en/"

except:
    print("\tPlease provide 'Hindi' or 'English' after the command. Exit and run again.")
    input()
    exit()

os.makedirs(destination_path, exist_ok=True)
os.makedirs(full_ds_path, exist_ok=True)

linkfile = open(file_name, "r")
links = linkfile.readlines()

i = 0
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers = {'User-Agent': user_agent}
k = 1

display_top_n = 5

for quote_page in links:
    quote_page = quote_page.strip()

    if quote_page.count('http') > 1:
        continue

    print(f"Processing link: {quote_page}")

    try:
        page = urllib.request.urlopen(myquote(quote_page))
    except Exception as e:
        print(e)
        input('wait1')
        continue

    if "FILE_TERMINATES_HERE" in quote_page:
        break

    rstring = ''.join(random.choices(string.ascii_uppercase, k=4))
    dtstring = os.path.join(destination_path, re.search(r'\d+$', quote_page)[0] + rstring + ".txt")
    dtstring2 = os.path.join(full_ds_path, re.search(r'\d+$', quote_page)[0] + rstring + ".txt")

    file = open(dtstring, 'w')
    article_len = 0

    try:
        soup = BeautifulSoup(page, 'html.parser')
        selected_div = None
        for div in soup.findAll('div'):
            if div.text and 'read more at' in div.find(text=True):
                selected_div = div
                break
        urlink = selected_div.find("a", target="_blank")['href']
        request = urllib.request.Request(urlink, None, headers)
        response = urllib.request.urlopen(request)
        soup2 = BeautifulSoup(response, 'html.parser')
        file.write("#originalArticleHeadline" + "\n")

        if "hindustantimes.com" in urlink:
            hlist = soup2.find('h1')
            hlist2 = soup2.find('h2')
            file.write(hlist.text + "\n")
            file.write(hlist2.text + "\n")

            file.write("#originalArticleBody" + "\n")
            table = soup2.findAll('div', attrs={"class": "story-details"})
            for x in table:
                for p in x.findAll('p'):
                    file.write(p.text + "\n")

        # I Will Add more website-specific parsing logic here as needed

        else:
            hlist = soup2.find('h1')
            file.write(hlist.text + "\n")
            file.write("#originalArticleBody" + "\n")
            article = soup2.findAll('p')
            for a in article:
                file.write(a.text + "\n")
    except Exception as e:
        print('Abdullah Error', e)
        #input('wait')
        file.close()
        os.remove(dtstring)
        print('  --> Waiting of 1 sec. Press Ctrl+C to exit', dtstring, ' ' * 90, end='\r')
        time.sleep(1)
        continue

    file.write("\n")
    file.write("-" * 100 + "\n")
    file.write("#summaryHeadline\n" + soup.find("span", itemprop="headline").text.strip() + "\n")
    file.write("#summaryBody\n" + soup.find("div", itemprop="articleBody").text.strip() + "\n")
    file.write("#datePublished " + soup.find("span", itemprop="datePublished").text.strip() + " ")
    file.write(soup.find("span", clas="date").text.strip() + "\n")
    #file.write(soup.find("span", attrs={'class': 'short'}).text.strip() + " by " + soup.find("span", attrs={
        #'class': 'author'}).text.strip() + " from News inShorts\n")
    file.write("#reference_link: " + quote_page + "\n")
    file.write("#original_link: " + urlink)
    file.close()

    print('dtstring', dtstring)
    #input('wait')

    print(str(i) + " " + quote_page[29:-14] + "\n    Article and Summary pulled!" + " " + datetime.datetime.now().strftime(
        "%H:%M:%S") + ' ' * 100 + '\n    file ' + dtstring2 + ' ' + " " * 100 + '\n' + '-' * 50 + ' ' * 100)
    if (k % display_top_n == 0):
        print("\033[A" * (display_top_n * 4), end='')

    k += 1

print("\n" * 7 + str(k - 1) + " Articles and their summaries pulled!")


# Exploratory Code

In [ ]:
import urllib.request
import urllib
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import quote
import datetime
import os
import time
import sys
import random, string, re

In [ ]:
def encode_hindi_url(quote_page):
    url = urllib.parse.urlsplit(quote_page)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    return url

input_file = "links.txt"

try:
    file = open(input_file, "r", encoding="utf-8")
    links = file.readlines()
    file.close()
except FileNotFoundError:
    print(f"File '{input_file}' not found.")
    exit()


output_directory = "scraped_headings/"


if not os.path.exists(output_directory):
    os.makedirs(output_directory)


for i, link in enumerate(links[:5]):  # Only top 5 links
    link = link.strip()
    print(f"Scraping link {i + 1}: {link}")


    encoded_link = encode_hindi_url(link)

    try:
        page = urllib.request.urlopen(encoded_link)
    except Exception as e:
        print(f"Error fetching {encoded_link}: {e}")
        continue

    soup = BeautifulSoup(page, 'html.parser')


    heading = soup.find("span", itemprop="headline")

    if heading:
        heading_text = heading.text.strip()

        filename = os.path.join(output_directory, f"heading_{i + 1}.txt")

        with open(filename, "w", encoding="utf-8") as file:
            file.write(heading_text)

        print(f"Heading {i + 1} saved.")

    time.sleep(random.uniform(1, 3))

print(f"Scraping complete for the top 5 links. Headings saved in '{output_directory}'")

Scraping link 1: https://inshorts.com/hi/news/डेरा-सच्चा-सौदा-प्रमुख-गुरमीत-राम-रहीम-सिंह-अनुयायी-की-हत्या-के-लिए-दोषी-करार-1633677650409
Heading 1 saved.
Scraping link 2: https://inshorts.com/hi/news/सहारनपुर-में-बसपा-के-फज़लुर्रहमान-सबसे-आगे-मसूद-को-भी-मिल-चुके-सवा-लाख-वोट-1558601614502
Heading 2 saved.
Scraping link 3: https://inshorts.com/hi/news/गुरुग्राम-में-फ्लैट-के-नाम-पर-दिल्ली-के-व्यापारी-से-₹9125-की-ठगी-1556099842849
Heading 3 saved.
Scraping link 4: https://inshorts.com/hi/news/अपने-आत्मीय-मित्र-को-याद-कर-रहा-हूं-पासवान-की-बरसी-पर-उनके-परिवार-को-संदेश-में-पीएम-1631439024472
Heading 4 saved.
Scraping link 5: https://inshorts.com/hi/news/मुज़फ्फरनगर-में-प्रेम-प्रसंग-के-चलते-हमलावरों-ने-की-युवक-की-गोली-मारकर-हत्या-1559907245740
Heading 5 saved.
Scraping complete for the top 5 links. Headings saved in 'scraped_headings/'
